# Historical Prices With the OpenBB Platform

This notebook demonstrates some of the ways to approach loading historical price data using the OpenBB Platform.  The action is in the Equity module; but first, we need to initialize the notebook with the import statements block.

## Import Statements

In [1]:
from datetime import datetime, timedelta

import pandas as pd
from openbb import obb

## The Equity Module

Historical market prices typically come in the form of OHLC+V - open, high, low, close, volume.  There may be additional fields returned by a provider, but those are the expected columns.  Granularity and amount of historical data will vary by provider and subscription status.  Visit their websites to understand what your entitlements are.

### openbb.equity.price.historical()

- This endpoint has the most number of providers out of any function. At the time of writing, choices are:

['alpha_vantage', 'cboe', 'fmp', 'intrinio', 'polygon', 'tiingo', 'yfinance']

- Common parameters have been standardized across all souces, `start_date`, `end_date`, `interval`.

- The default interval will be `1d`.

- The depth of historical data and choices for granularity will vary by provider and subscription status.  Refer to the website and documentation of each source understand your specific entitlements.

- For demonstration purposes, we will use the `openbb-yfinance` data extension.

In [2]:
df_daily = obb.equity.price.historical(symbol="spy", provider="yfinance")
df_daily.to_df().head(1)

,open,high,low,close,volume,split_ratio,dividend,capital_gains
date,,,,,,,,
2023-08-22,441.179993,441.179993,437.570007,438.149994,65062900,0.0,0.0,0.0


To load the entire history available from a source, pick a starting date well beyond what it might be. For example, `1900-01-01`

In [3]:
df_daily = obb.equity.price.historical(
    symbol="spy", start_date="1990-01-01", provider="yfinance"
).to_df()
df_daily.head(1)

,open,high,low,close,volume,split_ratio,dividend,capital_gains
date,,,,,,,,
1993-01-29,43.96875,43.96875,43.75,43.9375,1003200,0.0,0.0,0.0


#### Intervals

The intervals are entered according to this pattern:

- `1m` = One Minute
- `1h` = One Hour
- `1d` = One Day
- `1W` = One Week
- `1M` = One Month

The date for monthly value is the first or last, depending on the provider.  This can be easily resampled from daily data.

In [4]:
df_monthly = obb.equity.price.historical(
    "spy", start_date="1990-01-01", interval="1M", provider="yfinance"
).to_df()
df_monthly.tail(2)

,open,high,low,close,volume,split_ratio,dividend,capital_gains
date,,,,,,,,
2024-07-01,545.630005,565.159973,537.450012,550.809998,1038465500,0.0,0.0,0.0
2024-08-01,552.570007,563.150024,510.269989,556.570007,954486073,0.0,0.0,0.0


#### Resample a Time Series

`yfinance` returns the monthly data for the first day of each month.  Let's resample it to take from the last, using the daily information captured in the previous cells.

In [6]:
df_daily.index = pd.to_datetime(df_daily.index)
(
    df_daily[["open", "high", "low", "close", "volume"]]
    .resample("ME")
    .agg(
        {"open": "first", "high": "max", "low": "min", "close": "last", "volume": "sum"}
    )
)

,open,high,low,close,volume
date,,,,,
1993-01-31,43.968750,43.968750,43.750000,43.937500,1003200
1993-02-28,43.968750,45.125000,42.812500,44.406250,5417600
1993-03-31,44.562500,45.843750,44.218750,45.187500,3019200
1993-04-30,45.250000,45.250000,43.281250,44.031250,2697200
1993-05-31,44.093750,45.656250,43.843750,45.218750,1808000
...,...,...,...,...,...
2024-04-30,523.830017,524.380005,493.859985,501.980011,1592974000
2024-05-31,501.380005,533.070007,499.549988,527.369995,1153264400
2024-06-30,529.020020,550.280029,522.599976,544.219971,888923200


The block below packs an object with most intervals.

In [8]:
class HistoricalPrices:
    def __init__(self, symbol, start_date, end_date, provider, **kwargs) -> None:
        self.one: pd.DataFrame = (
            obb.equity.price.historical(
                symbol=symbol,
                start_date=start_date,
                end_date=end_date,
                interval="1m",
                provider=provider,
                **kwargs
            )
            .to_df()
            .convert_dtypes()
        )
        self.five: pd.DataFrame = (
            obb.equity.price.historical(
                symbol=symbol,
                start_date=start_date,
                end_date=end_date,
                interval="5m",
                provider=provider,
                **kwargs
            )
            .to_df()
            .convert_dtypes()
        )
        self.fifteen: pd.DataFrame = (
            obb.equity.price.historical(
                symbol=symbol,
                start_date=start_date,
                end_date=end_date,
                interval="15m",
                provider=provider,
                **kwargs
            )
            .to_df()
            .convert_dtypes()
        )
        self.thirty: pd.DataFrame = (
            obb.equity.price.historical(
                symbol=symbol,
                start_date=start_date,
                end_date=end_date,
                interval="30m",
                provider=provider,
                **kwargs
            )
            .to_df()
            .convert_dtypes()
        )
        self.sixty: pd.DataFrame = (
            obb.equity.price.historical(
                symbol=symbol,
                start_date=start_date,
                end_date=end_date,
                interval="60m",
                provider=provider,
                **kwargs
            )
            .to_df()
            .convert_dtypes()
        )
        self.daily: pd.DataFrame = (
            obb.equity.price.historical(
                symbol=symbol,
                start_date=start_date,
                end_date=end_date,
                interval="1d",
                provider=provider,
                **kwargs
            )
            .to_df()
            .convert_dtypes()
        )
        self.weekly: pd.DataFrame = (
            obb.equity.price.historical(
                symbol=symbol,
                start_date=start_date,
                end_date=end_date,
                interval="1W",
                provider=provider,
                **kwargs
            )
            .to_df()
            .convert_dtypes()
        )
        self.monthly: pd.DataFrame = (
            obb.equity.price.historical(
                symbol=symbol,
                start_date=start_date,
                end_date=end_date,
                interval="1M",
                provider=provider,
                **kwargs
            )
            .to_df()
            .convert_dtypes()
        )


def load_historical(
    symbol: str = "", start_date=None, end_date=None, provider=None, **kwargs
) -> HistoricalPrices:

    if symbol == "":
        display("Please enter a ticker symbol")
    if provider is None:
        provider = "yfinance"
    prices = HistoricalPrices(symbol, start_date, end_date, provider, **kwargs)

    return prices


prices = load_historical("spy")
display(prices.__dict__.keys())
display(prices.weekly.tail(2))

display(prices.one.head(2))

dict_keys(['one', 'five', 'fifteen', 'thirty', 'sixty', 'daily', 'weekly', 'monthly'])

,open,high,low,close,volume,split_ratio,dividend,capital_gains
date,,,,,,,,
2024-08-12,534.210022,555.02002,530.950012,554.309998,242599600,0,0.0,0
2024-08-19,554.72998,563.150024,553.859985,557.031006,142159243,0,0.0,0


,open,high,low,close,volume,split_ratio,dividend,capital_gains
date,,,,,,,,
2024-08-16 09:30:00,551.419983,551.929993,551.289978,551.349976,1881026,0,0,0
2024-08-16 09:31:00,551.349976,551.77002,551.26001,551.630005,230595,0,0,0


To demonstrate the difference between sources, let's compare values for daily volume from several sources.

In [11]:
# Collect the data

yahoo = obb.equity.price.historical("spy", provider="yfinance").to_df()
alphavantage = obb.equity.price.historical("spy", provider="alpha_vantage").to_df()
intrinio = obb.equity.price.historical("spy", provider="intrinio").to_df()
fmp = obb.equity.price.historical("spy", provider="fmp").to_df()
polygon = obb.equity.price.historical("spy", provider="polygon").to_df()

# Make a new DataFrame with just the volume columns
compare = pd.DataFrame()
compare["AV Volume"] = alphavantage["volume"].tail(10)
compare["FMP Volume"] = fmp["volume"].tail(10)
compare["Intrinio Volume"] = intrinio["volume"].tail(10)
compare["Yahoo Volume"] = yahoo["volume"].tail(10)
compare["Polygon Volume"] = polygon["volume"].tail(10)

compare.dropna(how="any")

,AV Volume,FMP Volume,Intrinio Volume,Yahoo Volume,Polygon Volume
date,,,,,
2024-08-09,45619558,45619558.0,45619558.0,45619600.0,45425963.0
2024-08-12,42542069,42542069.0,42542069.0,42542100.0,42533175.0
2024-08-13,52333073,52333073.0,52333073.0,52333100.0,50110167.0
2024-08-14,42446929,42446929.0,42446929.0,42446900.0,42362522.0
2024-08-15,60846812,60846812.0,60846812.0,60846800.0,60762738.0
2024-08-16,44430728,44430728.0,44430728.0,44430700.0,44368969.0
2024-08-19,39121793,39121793.0,39121793.0,39121800.0,38648958.0
2024-08-20,33732264,33732264.0,33732264.0,33732300.0,33693989.0
2024-08-21,41514600,38682509.0,41514600.0,41467000.0,41532360.0


## Other Types of Symbols

Other types of assets and ticker symbols can be loaded from `obb.equity.price.historical()`, below are some examples but not an exhaustive list.

### Share Classes

Some sources use `-` as the distinction between a share class, e.g., `BRK-A` and `BRK-B`. Other formats include:

- A period: `BRK.A`
- A slash: `BRK/A`
- No separator, the share class becomes the fourth or fifth letter.

```python
obb.equity.price.historical("brk.b", provider="polygon")
```

```python
obb.equity.price.historical("brk-b", provider="fmp")
```

While some providers handle the different formats on their end, others do not. This is something to consider when no results are returned from one source. Some may even use a combination, or accept multiple variations. Sometimes there is no real logic behind the additional characters, `GOOGL` vs. `GOOG`. These are known unknown variables of ticker symbology, what's good for one source may return errors from another. 

### Regional Identifiers

With providers supporting market data from multiple jurisdictions, the most common method for requesting data outside of US-listings is to append a suffix to the ticker symbol (e.g., `RELIANCE.NS` for Indian equities). Formats may be unique to a provider, so it is best to review the source's documentation for an overview of their specific conventions. [This page](https://help.yahoo.com/kb/SLN2310.html) on Yahoo describes how they format symbols, which many others follow to some degree.

### Indexes

Sources will have their own treatment of these symbols, some examples are:

- YahooFinance/FMP/CBOE: ^RUT
- Polygon: I:NDX

### Currencies

FX symbols face the same dilemna as share classes, there are several variations of the same symbol.

- YahooFinance: `EURUSD=X`
- Polygon: `C:EURUSD`
- AlphaVantage/FMP: `EURUSD`

**The symbol prefixes are handled internally when `obb.currency.price.historical()` is used to enter a pair with no extra characters.**

### Crypto

Similar, but different to FX tickers.

- YahooFinance: `BTC-USD`
- Polygon: `X:BTCUSD`
- AlphaVantage/FMP: `BTCUSD`

**The symbol prefixes are handled internally when `obb.crypto.price.historical()` is used to enter a pair with no extra characters and placing the fiat currency second.**

### Futures

Historical prices for active contracts, and the continuation chart, can be fetched via `yfinance`.

- Continuous front-month: `CL=F`
- December 2023 contract: `CLZ24.NYM`
- March 2024 contract: `CLH24.NYM`

Individual contracts will require knowing which of the CME venues the future is listed on. `["NYM", "NYB", "CME", "CBT"]`.

### Options

Individual options contracts are also loadable from `openbb.equity.price.historical()`.

- YahooFinance: `SPY241220P00400000`
- Polygon: `O:SPY241220P00400000`

These examples represent only a few methods for fetching historical price data.  Explore the contents of each module to find more!

In [13]:
obb.equity.price.historical("SPY251219P00400000", provider="yfinance").to_df()

,open,high,low,close,volume,split_ratio,dividend
date,,,,,,,
2023-08-22,25.10,25.100000,25.10,25.100000,11,0.0,0.0
2023-08-23,25.00,25.000000,24.50,24.500000,2,0.0,0.0
2023-08-24,25.00,25.200001,25.00,25.200001,2,0.0,0.0
2023-08-25,25.35,25.350000,24.18,24.549999,0,0.0,0.0
2023-08-29,24.00,24.700001,22.50,23.910000,0,0.0,0.0
...,...,...,...,...,...,...,...
2024-08-16,5.95,6.100000,5.95,5.990000,4,0.0,0.0
2024-08-19,5.92,5.920000,5.71,5.710000,40,0.0,0.0
2024-08-20,5.73,6.240000,5.73,6.240000,42,0.0,0.0


In [16]:
obb.equity.price.historical("SPX", provider="cboe").to_df()

,open,high,low,close,volume
date,,,,,
1978-01-03,94.74,95.15,93.49,93.82,0
1978-01-04,93.16,94.10,92.57,93.52,0
1978-01-05,94.18,94.53,92.51,92.74,0
1978-01-06,92.06,92.66,91.05,91.62,0
1978-01-09,90.82,91.48,89.97,90.64,0
...,...,...,...,...,...
2024-08-15,5501.13,5546.23,5501.13,5543.22,0
2024-08-16,5530.50,5561.98,5525.17,5554.25,0
2024-08-19,5557.23,5608.30,5550.74,5608.25,0


In [17]:
obb.equity.price.historical("^SPX", provider="fmp").to_df()

,open,high,low,close,volume,vwap,adj_close,unadjusted_volume,change,change_percent
date,,,,,,,,,,
2023-08-22,4415.33008,4418.58984,4382.77002,4387.54980,3522760000,4396.30,4387.54980,3.522760e+09,-27.78028,-0.006292
2023-08-23,4396.43994,4443.18018,4396.43994,4436.00977,3837270000,4425.21,4436.00977,3.837270e+09,39.56983,0.009000
2023-08-24,4455.16016,4458.29980,4375.54980,4376.31006,3723470000,4403.39,4376.31006,3.723470e+09,-78.85010,-0.017700
2023-08-25,4389.37988,4418.45996,4356.29004,4405.70996,3296180000,4393.49,4405.70996,3.296180e+09,16.33008,0.003720
2023-08-28,4426.02979,4439.56006,4414.97998,4433.31006,2957230000,4429.28,4433.31006,2.957230e+09,7.28027,0.001645
...,...,...,...,...,...,...,...,...,...,...
2024-08-16,5530.50000,5561.97998,5525.16992,5554.25000,3357690000,5542.97,5554.25000,3.357690e+09,23.75000,0.004294
2024-08-19,5557.22998,5608.29981,5550.74023,5608.25000,3222050000,5581.13,5608.25000,3.222050e+09,51.02002,0.009181
2024-08-20,5602.87988,5620.50977,5585.50000,5597.12012,2994420000,5601.50,5597.12012,2.994420e+09,-5.75976,-0.001028


In [19]:
obb.equity.price.historical("CLZ25.NYM", provider="yfinance").to_df()

,open,high,low,close,volume,split_ratio,dividend
date,,,,,,,
2023-08-22,71.419998,71.949997,71.029999,71.160004,5342,0.0,0.0
2023-08-23,71.120003,71.320000,69.709999,70.730003,5139,0.0,0.0
2023-08-24,70.459999,70.860001,69.870003,70.190002,7594,0.0,0.0
2023-08-25,70.050003,70.889999,69.470001,70.680000,9328,0.0,0.0
2023-08-28,70.690002,71.190002,70.239998,70.480003,6234,0.0,0.0
...,...,...,...,...,...,...,...
2024-08-16,70.940002,70.989998,69.440002,70.019997,38165,0.0,0.0
2024-08-19,70.099998,70.400002,68.870003,69.029999,29067,0.0,0.0
2024-08-20,69.150002,69.250000,68.309998,68.389999,29827,0.0,0.0


In [20]:
obb.equity.price.historical("CL=F", provider="fmp").to_df()

,open,high,low,close,volume,vwap,adj_close,unadjusted_volume,change,change_percent
date,,,,,,,,,,
2023-08-22,80.80,80.99,80.10,80.35,287489,80.48,80.35,287489.0,-0.45,-0.005569
2023-08-23,79.64,79.91,77.62,78.89,378146,78.81,78.89,378146.0,-0.75,-0.009417
2023-08-24,78.57,79.28,77.59,79.05,349230,78.64,79.05,349230.0,0.48,0.006109
2023-08-25,78.88,80.45,78.14,79.83,411409,79.47,79.83,411409.0,0.95,0.012000
2023-08-28,80.15,80.87,79.61,80.10,246584,80.19,80.10,246584.0,-0.05,-0.000624
...,...,...,...,...,...,...,...,...,...,...
2024-08-18,76.58,76.71,76.48,76.71,175,76.62,76.71,175.0,0.13,0.001698
2024-08-19,76.58,76.87,74.17,74.37,118172,75.50,74.37,118172.0,-2.21,-0.028900
2024-08-20,74.34,75.03,73.50,74.04,118172,74.23,74.04,118172.0,-0.30,-0.004036


In [21]:
obb.equity.price.historical("usdjpy=x", provider="yfinance").to_df()

,open,high,low,close,volume,split_ratio,dividend
date,,,,,,,
2023-08-22,146.238007,146.389999,145.501999,146.238007,0,0.0,0.0
2023-08-23,145.763000,145.813004,144.580002,145.763000,0,0.0,0.0
2023-08-24,144.673004,145.947006,144.621002,144.673004,0,0.0,0.0
2023-08-25,146.067001,146.604996,145.733994,146.067001,0,0.0,0.0
2023-08-28,146.531006,146.716003,146.278000,146.531006,0,0.0,0.0
...,...,...,...,...,...,...,...
2024-08-16,149.222000,149.229996,147.639008,149.222000,0,0.0,0.0
2024-08-19,147.955994,147.959000,145.220993,147.955994,0,0.0,0.0
2024-08-20,146.699005,147.319000,145.533997,146.699005,0,0.0,0.0


In [22]:
obb.currency.price.historical("usdjpy", provider="yfinance").to_df()

,open,high,low,close,volume
date,,,,,
2023-08-22,146.238007,146.389999,145.501999,146.238007,0.0
2023-08-23,145.763000,145.813004,144.580002,145.763000,0.0
2023-08-24,144.673004,145.947006,144.621002,144.673004,0.0
2023-08-25,146.067001,146.604996,145.733994,146.067001,0.0
2023-08-28,146.531006,146.716003,146.278000,146.531006,0.0
...,...,...,...,...,...
2024-08-16,149.222000,149.229996,147.639008,149.222000,0.0
2024-08-19,147.955994,147.959000,145.220993,147.955994,0.0
2024-08-20,146.699005,147.319000,145.533997,146.699005,0.0
